# Data Collection
We limit our scope to the top 10 pools (in terms of TVL).

In [1]:
from pprint import pprint

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

from pyammanalysis import run_query, uniswap_v3_subgraph_url

In [2]:
most_liquid_pool_ids_query = """
{
  pools(first: 10, orderBy: liquidity, orderDirection: desc) {
    id
  }
}
"""

most_liquid_pool_ids = run_query(uniswap_v3_subgraph_url(), most_liquid_pool_ids_query)
pprint(most_liquid_pool_ids)

{'data': {'pools': [{'id': '0x2e5848efcfac935dd243c9094048ac346e198e1d'},
                    {'id': '0x06bd1af522f43bc270203e96a019b4779195b870'},
                    {'id': '0x18bb49dabec36c29af9450f420e675f46c416042'},
                    {'id': '0xad76da68a204fe301cc9b1c60f7166db73d35e76'},
                    {'id': '0x058e136bc597082589d2cf5d0d021e22b6556a9c'},
                    {'id': '0x2df3f8e466d4924d86c35213dcef8c6955c6813a'},
                    {'id': '0xabb670b47096728571672408057ac258176c3809'},
                    {'id': '0xedcfece1060e568e10425e95e35b179d5b5a2b5f'},
                    {'id': '0xe744f5e2edfdcb9fdb43b288ecb8b21c8487e888'},
                    {'id': '0x235c74836bf329a8c4172af9d07173f38026a49c'}]}}


In [3]:
most_liquid_pool_addrs = list(
    map(lambda x: x["id"], most_liquid_pool_ids["data"]["pools"])
)
most_liquid_pool_addrs

['0x2e5848efcfac935dd243c9094048ac346e198e1d',
 '0x06bd1af522f43bc270203e96a019b4779195b870',
 '0x18bb49dabec36c29af9450f420e675f46c416042',
 '0xad76da68a204fe301cc9b1c60f7166db73d35e76',
 '0x058e136bc597082589d2cf5d0d021e22b6556a9c',
 '0x2df3f8e466d4924d86c35213dcef8c6955c6813a',
 '0xabb670b47096728571672408057ac258176c3809',
 '0xedcfece1060e568e10425e95e35b179d5b5a2b5f',
 '0xe744f5e2edfdcb9fdb43b288ecb8b21c8487e888',
 '0x235c74836bf329a8c4172af9d07173f38026a49c']

In [4]:
async def get_pool_by_id(id: str):
    transport = AIOHTTPTransport(url=uniswap_v3_subgraph_url())

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(
            """
            query getPoolById($pool_addr: ID!) {
                pool(id: $pool_addr) {
                    tick
                    token0 {
                        symbol
                        id
                        decimals
                    }
                    token1 {
                        symbol
                        id
                        decimals
                    }
                    feeTier
                    sqrtPrice
                    liquidity
                }
            }
            """
        )
        params = {"pool_addr": id}
        result = await session.execute(query, variable_values=params)
        pprint(result)


for pool_addr in most_liquid_pool_addrs:
    await get_pool_by_id(pool_addr)

{'pool': {'feeTier': '3000',
          'liquidity': '397207105940493915986914707343',
          'sqrtPrice': '61395992977184499289169498658',
          'tick': '-5100',
          'token0': {'decimals': '18',
                     'id': '0x293b0cd0991db07c8529febb01bc7d052315c5ab',
                     'symbol': 'ITO'},
          'token1': {'decimals': '18',
                     'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                     'symbol': 'WETH'}}}
{'pool': {'feeTier': '3000',
          'liquidity': '370222054922916232567263130009',
          'sqrtPrice': '79872537192210627084928868',
          'tick': '-138000',
          'token0': {'decimals': '18',
                     'id': '0x397cc4dfb5528dc31f73f55eb738ad7aa89651a4',
                     'symbol': 'HVC'},
          'token1': {'decimals': '18',
                     'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                     'symbol': 'WETH'}}}
{'pool': {'feeTier': '3000',
          'liquidity': '36